## Deployment of KG UI

#1. Get Access to Dokku

Steps: https://dokku.dev.maayanlab.cloud/user/10-access.html.
Request access from Daniel Clarke and provide him with your GitHub handle.

#2. Create a Dump of your Neo4j DB


First make sure your container is running on Neo4j

In [ ]:
docker-compose stop neo4j
docker-compose run neo4j neo4j-admin database dump neo4j
docker-compose start neo4j

This creates the dump in binary in "KnowledgeGraphDemo/data/neo4j/dumps/neo4j.dump"

#3. Create Neo4j on Dokku

In [ ]:
dokku neo4j:create <db-name> neo4j 5.19

#4. Ingest your Dump

In [ ]:
cd data/neo4j/dumps
dokku neo4j:import <db-name> < neo4j.dump

#5. Create your App

In [ ]:
dokku apps:create <app-name>

#6. Get the Credentials and Update your .env File in Gene-Knowledge-Graph

In [ ]:
dokku neo4j:info <app-name> --dsn

This will return an output in this format: bolt://neo4j:[new_password]@[new_url]

Update your .env file in Gene-Knowledge-Graph like so

NEXT_PUBLIC_PREFIX=\
NEXT_PUBLIC_NEO4J_DEV_URL=bolt://localhost:7687\
NEXT_PUBLIC_NEO4J_URL=bolt://[new_url]\
NEXT_PUBLIC_NEO4J_V5_URL=bolt://[new_url] \
NEO4J_DEV_URL=bolt://localhost:7687

NEXT_PUBLIC_NEO4J_USER=neo4j
NEXT_PUBLIC_NEO4J_NAME=kg


NEXT_PUBLIC_NEO4J_PASSWORD=[new_password]



NEXT_PUBLIC_DOMAINS=minio.dokku.maayanlab.cloud,github.com,minio.dev.maayanlab.cloud,s3.amazonaws.com,raw.githubusercontent.com
NEXT_PUBLIC_ENRICHR_URL=https://maayanlab.cloud/Enrichr\
NEXT_PUBLIC_GENESHOT_URL=https://maayanlab.cloud/geneshot\
NEXT_PUBLIC_HOST=https://[app_name].dev.maayanlab.cloud/\
NEXT_PUBLIC_SCHEMA=\
NEXT_PUBLIC_TURL=turlkey\
NEXT_PUBLIC_TURL_URL=https://maayanlab.cloud/turl/api/register\
APP_NAME=[app_name]\
DOCKERHUB_NAME=[dockerhub_name]\
NEO4J_VERSION=5
NEO4J_USER=

#6. Update Dokku Config

Run this command:  dokku config:set <app_name> "NEXT_PUBLIC_NEO4J_DEV_URL=bolt://localhost:7687". Make sure to add every key-value pair in your .env file like how we did for the NEXT_PUBLIC_NEO4J_DEV_URL and it's value. It should be a long command.

#7. Build the Site

In [ ]:
cd Gene-Knowledge-Graph
npm version patch

#8. Deploy Directly from DockerHub

In [ ]:
dokku git:from-image <app_name> <image_name>

Check docker-compose.yml in Gene-Knowledge-Graph and look for "services>kg>image" for the image name. Make sure to include the version at the end of the image. This version will update each time you run npm version patch so make sure to update the git command as well.

#9. Finalizing Deployment

In [ ]:
dokku proxy:clear-config <app_name>
dokku proxy:ports-set <app_name> 'http:80:<port_number>' // Ex. 3000 if your local site was at localhost:3000
dokku traefik:enable <app_name>
dokku neo4j:link <db_name> <app_name>

And that's all! Check your site at https://[app_name].dev.maayanlab.cloud/

#10. What to do if you make changes and want to redeploy

In [ ]:
cd Gene-Knowledge-Graph
npm version patch
dokku git:from-image <app_name> <image_name> // remember to get the new version from docker-compose.yml